## 6.4 Working with geolocation data


In this practical, we will look at the location data we have stored for each tweet.  We will learn how to process geolocation coordinates, and obtain a more human readable label for these locations by obtaining full address information such as town, city, and county names, which we will be using to summarise and visualise the tweets.

Our first task this week is to load our data set and beginning exploring the locations referenced in the Twitter data set.  We import the Pandas library as before, read in the .csv file, and call .head(20) on the data frame to view the first 20 rows of data:

In [1]:
import pandas as pd

df = pd.read_csv('./data/twitter_data_olio_UK_b.csv')
df.head()

date                  _id   place_name  \
0  Tue Sep 29 20:57:35 +0000 2020  1311047494021701632        Oadby   
1  Tue Sep 29 20:43:53 +0000 2020  1311044046471585792        Oadby   
2  Tue Sep 29 20:43:26 +0000 2020  1311043935322611712  Hammersmith   
3  Tue Sep 29 20:41:30 +0000 2020  1311043448972083203      Barking   
4  Tue Sep 29 20:41:06 +0000 2020  1311043346257772544      Glasgow   

  place_type      lat      lng  \
0       city  52.5883 -1.09612   
1       city  52.5883 -1.09612   
2       city  51.4865 -0.22651   
3       city  51.5409  0.12200   
4       city  55.8720 -4.26896   

                                                text  
0  "Hovis granary wholemeal" in #Leicester #Unite...  
1  "Celery" in #Leicester #UnitedKingdom https://...  
2  "Eat 17 - small baguettes" in #London #UnitedK...  
3  "Salad" in #Dagenham #UnitedKingdom https://t....  
4  "Humous and Chipotle Wrap" in #Glasgow #United...

The next step is to look at summary statistics computed over the whole dataset, the .describe() method produces a simple report on the number of rows of data, the number of unique items we have, and the most common category together with its count. 

In [3]:
df['place_name'].describe()

count         4634
unique         289
top       East Ham
freq           241
Name: place_name, dtype: object

From running the example, we see that the dataset contains 4,634 unique tweets, with 289 unique locations mentioned across the United Kingdom. Of the total tweets the most mentioned location is 'East Ham' with 241 tweets.  To learn more about the whole data set, we can summarise the values of our columns. We will first explore the locations that are mentioned with the tweets, by grouping the data according to the locations stored under the 'place_name' column. We then select the top-10 rows with the highest frequency counts as our summary:

In [4]:
place_names = df.groupby(['place_name']).size().nlargest(10)
place_names

place_name
East Ham       241
Walthamstow    226
Richmond       188
Reading        176
Hull           148
Islington      145
Stevenage      129
Grays          114
Ipswich        108
Poplar         108
dtype: int64

Let's visualise these summary statistics to get a feel for the proportion of tweets mentioning each location. We plot our summary as a simple bar chart, and add a title and y-axis label:

In [6]:
barchart = place_names.plot(kind='bar')
barchart.set_title('Tweeted locations')
barchart.set_ylabel('Count')
barchart

Another column we can inspect is the 'place_type', which will tell us the types of locations mentioned, such as 'city'. Calling the .size() method on the grouped data will return the number of rows per categ

In [7]:
df.groupby(['place_type']).size()

place_type
admin      83
city     4551
dtype: int64

The majority of our data set contains locations relating to cities. However, a small proportion of our data (83 rows in total) are described by the Twitter API as 'admin'.  Let's explore this further to see what this means by selecting the subset of our data where the 'place_type' is equal to 'admin'. We will select a random sample of 20 rows from this subset to act as our summary:

In [8]:
df.loc[df['place_type'] == 'admin'].sample(10)

date                  _id     place_name  \
3381  Thu Oct 01 08:00:31 +0000 2020  1311576715274137600     South East   
955   Tue Sep 29 10:25:03 +0000 2020  1310888314506010624     South East   
3396  Thu Oct 01 07:55:39 +0000 2020  1311575489962749957     South East   
3403  Thu Oct 01 07:51:15 +0000 2020  1311574382779793408  West Midlands   
3382  Thu Oct 01 08:00:31 +0000 2020  1311576715207020544  West Midlands   
2951  Wed Sep 30 07:00:34 +0000 2020  1311199239511461888           East   
2595  Mon Sep 28 07:52:31 +0000 2020  1310487539543875584           East   
956   Tue Sep 29 10:24:57 +0000 2020  1310888288841072640     South East   
3389  Thu Oct 01 07:57:28 +0000 2020  1311575946558951426  West Midlands   
3397  Thu Oct 01 07:55:16 +0000 2020  1311575395536441345  West Midlands   

     place_type        lat       lng  \
3381      admin  51.454500 -0.856300   
955       admin  51.600200 -1.263760   
3396      admin  51.454500 -0.856300   
3403      admin  52.366900 -1.701310   
3382      admin  52.366900 -1.701310   
2951      admin  52.334200 -0.213470   
2595      admin  52.604352  1.172783   
956       admin  51.600200 -1.263760   
3389      admin  52.366900 -1.701310   
3397      admin  52.366900 -1.701310   

                                                   text  
3381  "Tiger  bloomer" in #Reading #UnitedKingdom ht...  
955   "Supersweet strawberries " in #Didcot #UnitedK...  
3396  "Orange and parsnip cake with pecans" in #Read...  
3403  " Baby spinach " in #Solihull #UnitedKingdom h...  
3382  "Watercress" in #Solihull #UnitedKingdom https...  
2951  "Shower foam " in #Huntingdon #UnitedKingdom h...  
2595  "5 sweet pastries" in #Norwich #UnitedKingdom ...  
956   "Supersweet strawberries " in #Didcot #UnitedK...  
3389  "Mixed vegetable meddley" in #Solihull #United...  
3397  "Butterhead salad " in #Solihull #UnitedKingdo...

From the summary we observe that the place type 'admin' appears to be assigned to large regions, in other words, the location encoded by the tweet relates to an administrative district rather than a city, despite the presence of hashtags relating to the city name in the tweet text, e.g. #Falkirk, #Didcot, and #Enfield. Before we continue we will correct the place name for these tweets so that they refer to the city rather than the level of administrative district.

How might we correct these values?  If we think about the information we have about each tweet, can you think of a method to retrieve the correct city name? We will correct this issue by extracting the missing values using another API that will identify a location and the full address given coordinates in the form of latitude and longitude, by applying a method known as reverse geo-coding. Given a set of coordinates, reverse geo-coding involves extracting a text string with address information that we can use to fill in our missing data. We will extract the town or city name for the locations listed as "admin" using the geo-coordinates latitude and longitude stored in the tweet.  

To do this we will import another Python library called geopy, which will take these and return a full address. You can learn more about using the geopy library by reading the official documentation (https://geopy.readthedocs.io/en/latest/). 

First, let's see how this works in practice. We first import the necessary tools and define a random name for our application, which will be used by the API to identify our request. This identifier can be any text string you can think of.  

Once the library is imported, we create a 'geolocator' object, which has a 'reverse' method where we pass the latitude and longitude coordinates. This method returns address details about the location, which we will store in a variable called 'location': 

In [9]:
# reverse geocoding
from geopy.geocoders import Nominatim

app_name = "my awesome app" # <- necessary to identify your request and access the geopy API.

geolocator = Nominatim(user_agent=app_name)

coordinates = '51.52194, -0.13032'

location = geolocator.reverse(coordinates)

We can now inspect the data returned by the method call. To view a text string of the full address, we can view the .address attribute:

In [10]:
# Retrieving the full address string:
location.address

'Birkbeck College, Malet Street, St Giles, Bloomsbury, London Borough of Camden, England, WC1E 7HX, United Kingdom'

We can also obtain a more useful representation that we can use to extract specific values from the address.  The returned location data also has a 'raw' attribute containing a JSON representation of the address and additional metadata:

In [11]:
# Retrieving a JSON object for querying specific attributes
location.raw

{'place_id': 93085019,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 28006263,
 'lat': '51.521766799999995',
 'lon': '-0.13019071705665525',
 'display_name': 'Birkbeck College, Malet Street, St Giles, Bloomsbury, London Borough of Camden, England, WC1E 7HX, United Kingdom',
 'address': {'amenity': 'Birkbeck College',
  'road': 'Malet Street',
  'neighbourhood': 'St Giles',
  'suburb': 'Bloomsbury',
  'city': 'London Borough of Camden',
  'state': 'England',
  'postcode': 'WC1E 7HX',
  'country': 'United Kingdom',
  'country_code': 'gb'},
 'boundingbox': ['51.5215222', '51.5221519', '-0.1306977', '-0.1297959']}

## Task: Locate yourself

As we can see, the .address attribute stores the address according to various fields, such as 'road', 'suburb', and 'city', which means we have all the information we need to address the issue of missing or incorrect values.  Let's check the address information for your current location.  First, visit <a src="https://mylocation.org/">mylocation.org</a> and make a note of the latitute and longitude coordinates. Paste these together and assign them to the 'coordinates' variable.  When you run the example again, you should see the full address of your current location.

In [12]:
# reverse geocoding
from geopy.geocoders import Nominatim

app_name = "my awesome app" # <- necessary to identify your request and access the geopy API.

geolocator = Nominatim(user_agent=app_name)

coordinates = 'YOUR LATITUDE, YOUR LONGITUDE' # <- paste your coordinates here
try:
    location = geolocator.reverse(coordinates)
    location
except: print("Don't forget, your coordinates should be formatted to look like this '3.0000,50.0000' ")

Don't forget, your coordinates should be formatted to look like this '3.0000,50.0000' 


Now let's return to the issue we wish to address, that is we want to extract the county name for areas defined as administrative ('admin'). One cavaet is that not all the missing values are of the type 'city', some locations maybe 'suburbs', or 'villages', so we need to capture this in our code to ensure that we fill in all the missing data correctly.  We first load the data we processed earlier, then proceed to iterate through each row and pull out the relevant values based on the conditions we have set, as follows:

In [15]:
print("Reverse geo-coding in progress, this may take a minute...")
# reverse geocoding
from geopy.geocoders import Nominatim

app_name = "my awesome app" # <- necessary to identify your request and gain access to the geopy API.

geolocator = Nominatim(user_agent=app_name)

for idx, lat, lng, place_type, text in zip(df.index, df.lat, df.lng, df.place_type, df.text):
    
    if place_type != 'admin': continue # <- skip all rows if the place_type is different from admin.
        
    location = geolocator.reverse("%s, %s" %(lat, lng))
    
    address = location.raw['address']
    
    place_name = address.get('city', None) # <- Check to see if there is a 'city' attribute we can access.
    place_type = 'city' # <- we will assume a default place type of city.
    
    if place_name == None:
        place_name = address.get('suburb', None) # <- No city attribute, check to see if there is a 'suburb' attribute we can access.
        place_type = 'suburb'
        
    if place_name == None:
        place_name = address.get('hamlet', None) # <- No city or suburb attribute, check to see if there is a 'suburb' attribute we can access.
        place_type = 'hamlet'   
        
    if place_name == None: 
        place_name = address.get('town', None) # <- No city, suburb, or hamlet attribute, check to see if there is a 'village' attribute we can access.
        place_type = 'town'
        
    if place_name == None: 
        place_name = address.get('village', None) # <- None of the above, check to see if there is a 'village' attribute we can access.
        place_type = 'village'
        
    df.loc[idx, 'place_name'] = place_name
    df.loc[idx, 'place_type'] = place_type
    
    print([idx], (lat, lng), place_name, place_type)

Reverse geo-coding in progress, this may take a minute.


Once complete, we can now inspect the unique values stored in the place_types column to verify that all rows with a place type of 'admin' have been replaced with more specific information:

In [14]:
df['place_type'].unique()

array(['city', 'suburb', 'village', 'hamlet'], dtype=object)

As we can see all of the locations listed as 'admin' by the Twitter API, have now been corrected using information we have gathered through reverse geo-coding.